In [1]:
import json
import urllib3
import pickle as pkl
from bs4 import BeautifulSoup

In [2]:
http = urllib3.PoolManager()
urllib3.disable_warnings()
WIKI = "https://fr.wikipedia.org/wiki/Canton_(Suisse)"
header = {'User-Agent': 'Mozilla/5.0'} #Needed to prevent 403 error on Wikipedia

In [3]:
r = http.request('GET', WIKI, headers = header)
soup = BeautifulSoup(r.data, "lxml")
table = soup.find("table", { "class" : "wikitable sortable" })

In [12]:
code = []
canton = []

for row in table.findAll("tr"):
    code_cells = row.findAll("th")
    if len(code_cells) == 2:
        code.append(str(code_cells[0].find(text=True)))
    canton_cells = row.findAll("td")
    if len(canton_cells) == 10:
        canton.append(str(canton_cells[0].find(text=True)))

In [13]:
code

['ZH',
 'BE',
 'LU',
 'UR',
 'SZ',
 'OW',
 'NW',
 'GL',
 'ZG',
 'FR',
 'SO',
 'BS',
 'BL',
 'SH',
 'AR',
 'AI',
 'SG',
 'GR',
 'AG',
 'TG',
 'TI',
 'VD',
 'VS',
 'NE',
 'GE',
 'JU',
 'CH']

In [14]:
canton

['Zurich',
 'Berne',
 'Lucerne',
 'Uri',
 'Schwytz',
 'Obwald',
 'Nidwald',
 'Glaris',
 'Zoug',
 'Fribourg',
 'Soleure',
 'Bâle-Ville',
 'Bâle-Campagne',
 'Schaffhouse',
 'Appenzell Rhodes-Extérieures',
 'Appenzell Rhodes-Intérieures',
 'Saint-Gall',
 'Grisons',
 'Argovie',
 'Thurgovie',
 'Tessin',
 'Vaud',
 'Valais',
 'Neuchâtel',
 'Genève',
 'Jura',
 'Suisse']

In [15]:
dico = dict(zip(canton, code))
pkl.dump(dico, open('Data/map_cantons.pkl', 'wb')) 